In [ ]:
@file:DependsOn("/data/tools/actin-personalization/actin-personalization.jar")

import com.hartwig.actin.personalization.ncr.serialization.NcrDataReader

val records = NcrDataReader.read("/data/patient_like_me/ncr/K2400223.csv")

In [ ]:
import com.hartwig.actin.personalization.datamodel.DistantMetastasesStatus
import com.hartwig.actin.personalization.datamodel.Episode
import com.hartwig.actin.personalization.datamodel.ReferencePatient
import com.hartwig.actin.personalization.datamodel.Treatment
import com.hartwig.actin.personalization.ncr.interpretation.ReferencePatientFactory

fun Episode.doesNotIncludeAdjuvantOrNeoadjuvantTreatment(): Boolean {
    return !hasHadPreSurgerySystemicChemotherapy &&
            !hasHadPostSurgerySystemicChemotherapy &&
            !hasHadPreSurgerySystemicTargetedTherapy &&
            !hasHadPostSurgerySystemicTargetedTherapy
}

val patients = ReferencePatientFactory.default().create(records)

val referencePop = patients.flatMap(ReferencePatient::tumorEntries).map { (diagnosis, episodes) ->
    diagnosis to episodes.single { it.order == 1 }
}
    .filter { (_, episode) ->
        episode.distantMetastasesStatus == DistantMetastasesStatus.AT_START &&
        // Comment out the next line (treatment filtering) to match R results:
        episode.systemicTreatmentPlan?.treatment?.let { it != Treatment.OTHER } == true &&
        episode.surgeries.isEmpty() &&
        episode.doesNotIncludeAdjuvantOrNeoadjuvantTreatment() &&
        episode.systemicTreatmentPlan?.observedPfsDays != null
    }
    .sortedBy { it.second.systemicTreatmentPlan!!.observedPfsDays!! }

val patientsByTreatment = referencePop.groupBy { (_, episode) ->
    episode.systemicTreatmentPlan!!.treatment.treatmentGroup
}
    .toList()
    .sortedByDescending { it.second.size }

In [ ]:
import com.hartwig.actin.personalization.similarity.population.DiagnosisAndEpisode

data class EventCountAndSurvivalAtTime(val daysSincePlanStart: Int, val numEvents: Int, val survival: Double)
    
tailrec fun eventAndCensorshipHistories(
    populationToProcess: List<DiagnosisAndEpisode>,
    eventHistory: List<EventCountAndSurvivalAtTime> = emptyList(),
    censorshipHistory: List<EventCountAndSurvivalAtTime> = emptyList()
): Pair<List<EventCountAndSurvivalAtTime>, List<EventCountAndSurvivalAtTime>> {
    return if (populationToProcess.isEmpty()) {
        eventHistory to censorshipHistory
    } else {
        val treatmentDetails = populationToProcess.first().second.systemicTreatmentPlan!!
        val previousEvent = eventHistory.lastOrNull() ?: EventCountAndSurvivalAtTime(0, 0, 1.0)
        val (newEventHistory, newCensorshipHistory) = if (treatmentDetails.hadProgressionEvent!!) {
            val newEvent = EventCountAndSurvivalAtTime(
                treatmentDetails.observedPfsDays!!, previousEvent.numEvents + 1, previousEvent.survival * (1 - (1.0 / populationToProcess.size))
            )
            Pair(eventHistory + newEvent, censorshipHistory)
        } else {
            val newCensorship = EventCountAndSurvivalAtTime(treatmentDetails.observedPfsDays!!, previousEvent.numEvents, previousEvent.survival)
            Pair(eventHistory, censorshipHistory + newCensorship)
        }
        eventAndCensorshipHistories(populationToProcess.drop(1), newEventHistory, newCensorshipHistory)
    }
}

val (eventHistory, censorshipHistory) = eventAndCensorshipHistories(referencePop)

In [ ]:
%use kandy

In [ ]:

plot {
    step {
        x(eventHistory.map(EventCountAndSurvivalAtTime::daysSincePlanStart), "days")
        y(eventHistory.map(EventCountAndSurvivalAtTime::survival), "survival")
    }
}

In [ ]:
import com.hartwig.actin.personalization.datamodel.LocationGroup
import com.hartwig.actin.personalization.similarity.population.PopulationDefinition

val popDefinitions = PopulationDefinition.createAllForPatientProfile(50, 1, true, setOf(LocationGroup.PERITONEUM))
val popHistories = popDefinitions.map { (name, criteria) ->
    val (eventHistory, censorshipHistory) = eventAndCensorshipHistories(referencePop.filter(criteria))
    Triple(name, eventHistory, censorshipHistory)
}
plot {
    step {
        x(popHistories.flatMap { (_, eventHistory, _) -> eventHistory.map(EventCountAndSurvivalAtTime::daysSincePlanStart) }, "days")
        y(popHistories.flatMap { (_, eventHistory, _) -> eventHistory.map(EventCountAndSurvivalAtTime::survival) }, "survival")
        color(popHistories.flatMap { (name, eventHistory, _) -> eventHistory.map { name } }, "population")
    }
}

In [ ]:
popHistories.map { (name, eventHistory, _) -> name to eventHistory.firstOrNull { it.survival <= 0.5 } }
    .forEach { (name, eventCountAndSurvivalAtTime) -> println("$name: ${eventCountAndSurvivalAtTime?.daysSincePlanStart} days") }

In [ ]:
val historiesByTreatment = referencePop.groupBy { (_, episode) -> episode.systemicTreatmentPlan!!.treatment.treatmentGroup.display }
    .mapValues { (_, tumors) -> eventAndCensorshipHistories(tumors) }
    .filter { (_, histories) -> histories.first.size >= 20 }
plot {
    step {
        x(historiesByTreatment.flatMap { (_, histories) -> histories.first.map(EventCountAndSurvivalAtTime::daysSincePlanStart) }, "days")
        y(historiesByTreatment.flatMap { (_, histories) -> histories.first.map(EventCountAndSurvivalAtTime::survival) }, "survival")
        color(historiesByTreatment.flatMap { (name, histories) -> histories.first.map { name } }, "treatment")
    }
}

In [ ]:
historiesByTreatment.mapValues { (_, histories) -> histories.first.first { it.survival <= 0.5 } }.entries
    .sortedByDescending { (_, medianEvent) -> medianEvent.daysSincePlanStart!! }
    .forEach { (name, eventCountAndSurvivalAtTime) -> println("$name: ${eventCountAndSurvivalAtTime?.daysSincePlanStart} days") }

In [ ]:
%use dataframe

fun pfsForPopulation(population: List<DiagnosisAndEpisode>): String {
    val medianPfs = eventAndCensorshipHistories(population).first.firstOrNull { it.survival <= 0.5 }?.daysSincePlanStart
    return "$medianPfs (n=${population.size})"
}

fun dataFrame(rowLabels: List<Any?>, firstColumnName: String, namedColumns: List<Pair<String, List<String>>>): DataFrame<*> {
    val labelStrings = rowLabels.map { it?.let { it.toString() } ?: "None" }
    return (listOf(firstColumnName to labelStrings) + namedColumns).toMap().toDataFrame()
}

fun pfsTable(patientsByTreatment: Map<String, List<DiagnosisAndEpisode>>, columnDefinitions: List<PopulationDefinition>): DataFrame<*> {
    val allPatients = patientsByTreatment.flatMap { it.value }
    val sortedPatients = patientsByTreatment.entries.sortedByDescending { it.value.size }
                             
    val entries = columnDefinitions.map { (title, criteria) ->
        val populationSize: Int = allPatients.count(criteria)
        val annotatedTitle = "$title (n=$populationSize)"
        annotatedTitle to sortedPatients.map { pfsForPopulation(it.value.filter(criteria)) }
    }
    
    return dataFrame(sortedPatients.map { it.key }, "Treatment", entries)
}

val patientsByTreatment = referencePop.groupBy { (_, episode) -> episode.systemicTreatmentPlan!!.treatment.treatmentGroup.display }
pfsTable(patientsByTreatment, popDefinitions)